# BOOK RECOMMENDATION SYSTEM

## What is a Recommendation System and Why is it important ?

![Image](books.jpg)



A recommendation system is a software algorithm or system that suggests items, products, or content to users based on their preferences, behavior, or similarities to other users. These systems are designed to provide personalized and relevant recommendations, aiming to enhance user experience, engagement, and satisfaction.

Recommendation systems are crucial in various domains, including e-commerce, entertainment, social media, and content streaming platforms

We are going to build a Book recommendation system which can recommend us a book to read based on what are other read and also what books were rated the highest

We are going to use two types of methods for our Recommendation System namely Collaborative and Content Based Filtering , on this particular we will be using Content based filtering

## Content Based Filtering

This filtering method uses item features to recommend other items similar to what the user likes and also based on their previous actions or explicit feedback. If we consider the example for our book recommender system, the additional information can be the **Title**, the **Ratings**, the **Genre** e.t.c 


### Import Packages

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib as plt
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
csv_book_mapping = {}
with open(r"C:\Users\PC\OneDrive\Documents\Movie Recommendation System\book_id_map.csv") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id , book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

Find users who likes books similar to ours

In [3]:
liked_books = ["4408","31147619","29983711","9401317","9317691","8153988","20494944"]

In [4]:
overlap_users = set()
with open(r"C:\Users\PC\OneDrive\Documents\Movie Recommendation System\goodreads_interactions.csv") as f:
    
    while True:
        line = f.readline()
        if not line:
            break
        user_id , csv_id , _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue
        try:
            rating = int(rating)
        except ValueError:
            continue
        book_id = csv_book_mapping[csv_id]
        if book_id in liked_books and rating > 4:
            overlap_users.add(user_id)


Finding what books those users liked

In [5]:
rec_line = []
with open(r"C:\Users\PC\OneDrive\Documents\Movie Recommendation System\goodreads_interactions.csv") as f:
    
    while True:
        line = f.readline()
        if not line:
            break
        user_id , csv_id , _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_line.append([user_id,book_id,rating])

In [6]:
recs = pd.DataFrame(rec_line,columns=['user_id','book_id','rating'])
recs['book_id'] = recs['book_id'].astype(str)

Let's view the top ten Recommended Books

In [7]:
top_recs = recs['book_id'].value_counts().head(10)
top_recs = top_recs.index.values

In [8]:
book_titles = pd.read_json("book_titles.json")

In [9]:
book_titles['book_id'] = book_titles['book_id'].astype(str)

In [10]:
book_titles.head()

,book_id,title,ratings,url,cover_image,title_clean
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook


Creating Initial Book Recommendations

In [11]:
book_titles[book_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,title_clean
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
463463,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
615314,5470,1984,2023937,https://www.goodreads.com/book/show/5470.1984,https://images.gr-assets.com/books/1348990566m...,1984
757376,38447,The Handmaid's Tale,648783,https://www.goodreads.com/book/show/38447.The_...,https://images.gr-assets.com/books/1498057733m...,the handmaids tale
790927,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
878151,18143977,All the Light We Cannot See,498685,https://www.goodreads.com/book/show/18143977-a...,https://images.gr-assets.com/books/1451445646m...,all the light we cannot see
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
902688,136251,Harry Potter and the Deathly Hallows (Harry Po...,1784684,https://www.goodreads.com/book/show/136251.Har...,https://images.gr-assets.com/books/1474171184m...,harry potter and the deathly hallows harry pot...
995137,15881,Harry Potter and the Chamber of Secrets (Harry...,1821802,https://www.goodreads.com/book/show/15881.Harr...,https://images.gr-assets.com/books/1474169725m...,harry potter and the chamber of secrets harry ...
1062354,29983711,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko


In [12]:
all_recs = recs['book_id'].value_counts()

In [13]:
all_recs = all_recs.to_frame().reset_index()

In [14]:
all_recs

,index,book_id
0,2767052,611
1,2657,603
2,3,590
3,4671,584
4,29983711,560
...,...,...
263838,15738189,1
263839,14896654,1
263840,15750046,1
263841,15761322,1


In [15]:
all_recs.columns = ["book_id","book_count"]

In [16]:
all_recs = all_recs.merge(book_titles ,how="inner",on="book_id")

In [17]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])

In [18]:
all_recs.sort_values("score" ,ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,title_clean,score
4,29983711,560,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko,38.426663
433,9317691,129,The Name of the Wind (The Kingkiller Chronicle...,1043,https://www.goodreads.com/book/show/9317691-th...,https://images.gr-assets.com/books/1360558233m...,the name of the wind the kingkiller chronicle 1,15.954938
145,4408,223,East of Eden,3447,https://www.goodreads.com/book/show/4408.East_...,https://images.gr-assets.com/books/1323882457m...,east of eden,14.426748
6486,26856502,18,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,9.257143
945,31147619,80,Homegoing,697,https://www.goodreads.com/book/show/31147619-h...,https://images.gr-assets.com/books/1491119004m...,homegoing,9.182209
4575,34927828,24,The Great Alone,70,https://www.goodreads.com/book/show/34927828-t...,https://images.gr-assets.com/books/1501852384m...,the great alone,8.228571
245,32920226,176,"Sing, Unburied, Sing",4592,https://www.goodreads.com/book/show/32920226-s...,https://images.gr-assets.com/books/1499340866m...,sing unburied sing,6.745645
507,21032488,116,"Doors of Stone (The Kingkiller Chronicle, #3)",2059,https://www.goodreads.com/book/show/21032488-d...,https://s.gr-assets.com/assets/nophoto/book/11...,doors of stone the kingkiller chronicle 3,6.535211
238,8153988,178,"The Eye of the World (Wheel of Time, #1)",5740,https://www.goodreads.com/book/show/8153988-th...,https://images.gr-assets.com/books/1465920672m...,the eye of the world wheel of time 1,5.519861
9996,26860699,12,"The Olympian Affair (The Cinder Spires, #2)",27,https://www.goodreads.com/book/show/26860699-t...,https://s.gr-assets.com/assets/nophoto/book/11...,the olympian affair the cinder spires 2,5.333333


In [19]:
popular_recs = all_recs[all_recs["book_count"] >75].sort_values("score",ascending=False)

From our previous search engine , we show recommended books to read based on what the previous views liked 

In [20]:
def show_book_img(val):
    return '<img src="{}" width=50></img>'.format(val)
popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'cover_image':show_book_img})

,book_id,book_count,title,ratings,url,cover_image,title_clean,score
245,32920226,176,"Sing, Unburied, Sing",4592,https://www.goodreads.com/book/show/32920226-sing-unburied-sing,,sing unburied sing,6.745645
507,21032488,116,"Doors of Stone (The Kingkiller Chronicle, #3)",2059,https://www.goodreads.com/book/show/21032488-doors-of-stone,,doors of stone the kingkiller chronicle 3,6.535211
261,30753987,170,The Leavers,5602,https://www.goodreads.com/book/show/30753987-the-leavers,,the leavers,5.158872
421,33253215,130,The Heart's Invisible Furies,3629,https://www.goodreads.com/book/show/33253215-the-heart-s-invisible-furies,,the hearts invisible furies,4.656930
708,33280160,98,What We Lose,2250,https://www.goodreads.com/book/show/33280160-what-we-lose,,what we lose,4.268444
292,17912498,159,The Queen of the Night,6115,https://www.goodreads.com/book/show/17912498-the-queen-of-the-night,,the queen of the night,4.134260
252,32283423,174,American War,7776,https://www.goodreads.com/book/show/32283423-american-war,,american war,3.893519
729,33621427,96,Home Fire,2390,https://www.goodreads.com/book/show/33621427-home-fire,,home fire,3.856067
70,30688435,286,Exit West,21378,https://www.goodreads.com/book/show/30688435-exit-west,,exit west,3.826176
224,26025588,181,Behold the Dreamers,8793,https://www.goodreads.com/book/show/26025588-behold-the-dreamers,,behold the dreamers,3.725805
